In [ ]:
import json
import numpy as np

import panel as pn
import plotly.graph_objects as go

pn.extension('plotly')

### About this App

The goals of this app are:

- Serve as an intermidiate level example of `Panel` package.
- Display a few `Panel` - `Plotly` interactions that are useful for data analysis.
- Crate a quick way of assessing results of 2D tracking algorithms.

### About the data

This is the result of a tracking algorithm that tracks centroid of cells in 2D as they move in space. The volume and the contour of the cell shape at each frame is stored in a JSON file. In total we have 6 different tracks with duration of 160 frames.

In [ ]:
# Read data
with open('tracks.json', 'r') as fj:
    full_data = json.load(fj)
data = full_data['data']
limits = full_data['limits']

In [ ]:
# Widgets
wg_ntracks = pn.widgets.IntSlider(
    name='Number of tracks',
    value=len(data),
    start=1,
    end=len(data)
)
wg_frames_interval = pn.widgets.RangeSlider(
    name='Frames',
    start=limits['Fmin'],
    end=limits['Fmax'],
    value=(limits['Fmin'], limits['Fmin']+10),
    step=1
)
wg_curr_track = pn.widgets.TextInput(
    name='Selected track:',
    value=str('0')
)
wg_display_delta = pn.widgets.Checkbox(
    name='Display volume variation',
    value=False
)
plot_timeseries = pn.pane.Plotly()
plot_contours = pn.pane.Plotly()
plot_distribution = pn.pane.Plotly()

In [ ]:
@pn.depends(wg_ntracks.param.value, wg_frames_interval.param.value, watch=True)
def get_timeseries(ntracks, frames_interval):
    ''' Displays volume of each cell in time as different traces.
    Hover over this plot to select a particular track. Everytime a
    new track is selected the cell volume distribution below is
    also updated and so does the contours of cell shape
    on the right side.
    '''
    fig = go.Figure()
    for i, track_info in enumerate(data):
        fig.add_trace(
            go.Scatter(
                x=track_info['frame'], y=track_info['volume'], mode='lines', name=f'track-{i}'
            )
        )
        if i >= ntracks-1:
            break
    fig.add_vrect(
        x0=frames_interval[0], x1=frames_interval[1],
        fillcolor="LightSalmon", opacity=0.5,
        layer="below", line_width=0,
    ),
    fig.update_xaxes(range=[0,limits['Fmax']], title={'text': 'Frame ID', 'font': {'size': 18}})
    fig.update_yaxes(title={'text': 'Cell Volume (#pixels)', 'font': {'size': 18}})
    fig.update_layout(showlegend=False, margin=go.layout.Margin(l=0,r=0,b=0,t=0))
    plot_timeseries.object = fig

@pn.depends(plot_timeseries.param.hover_data, watch=True)
def update_selected_track(hover_data):
    '''Updates the current selected track when hovering over
    the plot.
    '''
    if hover_data is not None:
        wg_curr_track.value = str(hover_data['points'][0]['curveNumber'])

@pn.depends(wg_curr_track.param.value, wg_frames_interval.param.value, watch=True)
def get_contour(track, frames_interval):
    ''' Display 2D cell shape contours of the selected track
    and in the range of frames selected by the interval selector.
    '''
    fo = int(frames_interval[0]) # Initial frame ID selected
    ff = int(frames_interval[1]) # Final frame ID selected
    # Available frames for current track
    f_available = data[int(track)]['frame']
    # Make sure frame ID in within available range
    f_init = np.min([f_available[-1], np.max([fo, f_available[0]])])
    f_last = np.max([f_available[0], np.min([ff, f_available[-1]])])
    fig = go.Figure()
    for f in range(fo,ff):
        # Convert frame ID from string to int
        fstr = str(f)
        if fstr in data[int(track)]['contour']:
            op = (f-f_init)/(f_last-f_init)
            ct = data[int(track)]['contour']
            fig.add_trace(
                go.Scatter(
                    x=ct[fstr]['x'], y=ct[fstr]['y'], mode='lines', opacity=op, line={'color': "black"}
                )
            )
    fig.update_xaxes(range=[0,1900], title={'text': 'X-coordinate', 'font': {'size': 18}})
    fig.update_yaxes(range=[0,1300], title={'text': 'Y-coordinate', 'font': {'size': 18}})
    fig.update_layout(showlegend=False, margin=go.layout.Margin(l=0,r=0,b=0,t=0))
    plot_contours.object = fig

@pn.depends(wg_curr_track.param.value, wg_display_delta.param.value, watch=True)
def get_distribution(track, display_delta):
    '''Display distribution of cell volume for the current
    track. Optionally user can display the distribution of the
    increment in cell volume.
    
    WARNING: The checkbox is not working properly when unchecked.
    '''
    z = data[int(track)]['volume']
    if display_delta:
        z = np.diff(z)
    fig = go.Figure()
    fig.add_trace(
        go.Histogram(x=z)
    )
    vrange = [limits['Vmin'], limits['Vmax']]
    title = 'Cell Volume (#pixels)'
    if display_delta:
        vrange = [-0.5*limits['Vmin'], 0.5*limits['Vmin']] # Hacky!
        title = 'Delta ' + title
    fig.update_xaxes(range=vrange, title={'text': title, 'font': {'size': 18}})
    fig.update_yaxes(title={'text': 'Frequency', 'font': {'size': 18}})

    fig.update_layout(showlegend=False, margin=go.layout.Margin(l=0,r=0,b=0,t=0))
    plot_distribution.object = fig

In [ ]:
# Make the plots
get_timeseries(wg_ntracks.value, wg_frames_interval.value)
get_distribution(wg_curr_track.value, wg_display_delta.value)
get_contour(wg_curr_track.value, wg_frames_interval.value)

# Put everything in a grid
gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)
gspec[:10, :10] = pn.Column(wg_ntracks,wg_frames_interval,plot_timeseries,wg_curr_track,wg_display_delta,plot_distribution)
gspec[:10, 10:20] = plot_contours
gspec.servable()